In [1]:
import requests
from bs4 import BeautifulSoup
import re
import threading
from datetime import datetime
import pandas as pd
import time
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

In [2]:
date='2023-11-01'
format=datetime.strptime(date, "%Y-%m-%d")
# print(format)
formatted_date_string = format.strftime("%d/%m/%Y")
print(formatted_date_string)

01/11/2023


In [25]:
# headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
#     }
# url = f"https://www.inews.id/indeks/all/01-11-2023/2"
# response = requests.get(url, headers=headers)
# soup = BeautifulSoup(response.text, 'html.parser')
# articles = soup.find_all('li', {"class": "padding-10px-all"})
# for article in articles:
#     link = article.find('a')['href']
#     print(link)


In [9]:
def scrape_links(date,page_number):
    format=datetime.strptime(date, "%Y-%m-%d")
# print(format)
    formatted_date_string = format.strftime("%d-%m-%Y")
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://www.inews.id/indeks/all/{formatted_date_string}/{page_number}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('li', {"class": "padding-10px-all"})
    links = []
    for article in articles:
        link = article.find('a')['href']
        links.append(link)
    print(f"Scraped {len(links)} links from page {page_number} url {url}")
    
    return links

In [10]:
def scrape_link_per_day(date, max_threads=5):
    page_number = 1
    page_links = []

    with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
        futures = []

        while True:
            future = executor.submit(scrape_links, date, page_number)
            futures.append(future)
            page_number += 1

            # Break the loop if no more articles are found
            if not future.result():
                break

        for future in concurrent.futures.as_completed(futures):
            page_links.extend(future.result())

    return page_links

In [11]:
link=scrape_link_per_day('2023-11-01')

Scraped 30 links from page 1 url https://www.inews.id/indeks/all/01-11-2023/1
Scraped 30 links from page 2 url https://www.inews.id/indeks/all/01-11-2023/2
Scraped 30 links from page 3 url https://www.inews.id/indeks/all/01-11-2023/3
Scraped 30 links from page 4 url https://www.inews.id/indeks/all/01-11-2023/4
Scraped 30 links from page 5 url https://www.inews.id/indeks/all/01-11-2023/5
Scraped 30 links from page 6 url https://www.inews.id/indeks/all/01-11-2023/6
Scraped 30 links from page 7 url https://www.inews.id/indeks/all/01-11-2023/7
Scraped 30 links from page 8 url https://www.inews.id/indeks/all/01-11-2023/8
Scraped 30 links from page 9 url https://www.inews.id/indeks/all/01-11-2023/9
Scraped 30 links from page 10 url https://www.inews.id/indeks/all/01-11-2023/10
Scraped 30 links from page 11 url https://www.inews.id/indeks/all/01-11-2023/11
Scraped 30 links from page 12 url https://www.inews.id/indeks/all/01-11-2023/12
Scraped 30 links from page 13 url https://www.inews.id/ind

In [73]:
def scrape_url(url,max_retries=2):
    retries = 0
    while retries < max_retries:
            try:
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
                }
                response = requests.get(url, headers=headers)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')
                    
                    
                    # Judul Berita
                    title_elem = soup.find('h1', {"style": "padding:0 0 10px; margin:0; font-weight:700; font-size:34px; color:#000;"})
                    if title_elem:
                        title_text = title_elem.text.strip()
                    else:
                        title_text = "Title not found"  
                    # tanggal berita
                    date_elem = soup.find('a', {"class": "author-profile"})
                    # print(date_elem)
                    if date_elem:
                        date_text= date_elem.text.strip()
                        date_text= date_text.replace('\n', '').replace('\r', '').replace('\t', '')
                        date_text=' '.join(date_text.split())
                        match = re.search(r'\b(\d{2} \w+ \d{4})', date_text)
                        if match:
                            extracted_date_str = match.group(1)
                        date_object = datetime.strptime(extracted_date_str, '%d %B %Y')
                        formatted_date = date_object.strftime('%Y-%m-%d')
                        # date_text = date_text
                    else:
                        date_text = "Date not found"
                    #     # Content Berita
                    body_elem = soup.find('div', {"itemprop": "articleBody"})
                        
                    if body_elem:
                        content_elem = body_elem.find_all('p')
                        content_text = ""
                        for p in content_elem:
                                content_text += p.text.strip() + "\n"
                            
                        if content_text.strip():
                            content_text=content_text
                            content_text = content_text.replace('\n', '').replace('\r', '').replace('\t', '')
                            content_text = ' '.join(content_text.split())
                        else:
                            content_text="Content not found"
                    else:
                            content_text="Content not found"

                    return{
                        'title': title_text,
                        'date': formatted_date,
                        'content':content_text,
                        'link' : url}
                elif response.status_code == 429:
                    print(f"Received a 429 error for {url}. Retrying in 5 seconds...")
                    time.sleep(5)
                else:
                    print(f"Failed to retrieve data from {url}: Status Code {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"Error fetching URL '{url}': {e}")
            except Exception as e:
                print(f"Error processing URL '{url}': {e}")
            retries += 1
            if retries < max_retries:
                print(f"Retrying {url} (Attempt {retries}/{max_retries})")
                time.sleep(5)  # You can adjust the delay as needed
    return None         

In [74]:
url='https://www.inews.id/news/nasional/relawan-garuda-dukung-ganjar-pranowo-mahfud-md-pilihan-terbaik'
data_inews = scrape_url(url)

In [75]:
print(data_inews)

{'title': 'Relawan Garuda Dukung Ganjar Pranowo-Mahfud MD: Pilihan Terbaik!', 'date': '2023-11-01', 'content': 'JAKARTA, iNews.id - Ratusan aktivis mahasiswa dan alumni dari Kelompok Cipayung yang tergabung dalam Relawan Garuda mendeklarasikan dukungan untuk pasangan Ganjar Pranowo dan Mahfud MD, Rabu (1/11/2023). Dukungan ditegaskan khususnya setelah muncul putusan Mahkamah Konstitusi (MK) yang kontroversial."Awal-awal kami masih bicara tentang dukungan pada Ganjar, tetapi kita tidak mau membentuk relawan karena sudah banyak relawan yang juga deklarasi. Tetapi tiba-tiba ada tsunami politik lewat operasi politik yang ada di MK, akhirnya kita tersentak," kata Koordinator Nasional Relawan Garuda Dikson Siringo-ringo di Yogyakarta, dikutip dari keterangan resmi.Terkait dengan moralitas dan etika politik, Dikson menilai pasangan Ganjar dan Mahfud merupakan tokoh yang menjunjung tinggi tentang kedua hal itu. Rekam jejak Ganjar dan Mahfud, kata Dikson, telah membuktikan itu.Menurut Dikson, G